# The Multi-Layer Perceptron Neural Network Classifier

The MLP classifier is a Neural Network algorithm that is capable of building complex models, efficiently used on large datasets (such as ours). The fact that efficiency is important for this large dataset, a MLP was thought of a good selection. This method of machine learning is sensitive to scaling so keep in mind that we will be scaling

In [1]:
# First things first, let's load our features in

In [2]:
# Import Data
# MUTATE DATAFRAMES ACCORDING TO THE EXPLORATORY DATA ANALYSIS CODE

#For data Manipulation
import numpy as np
import pandas as pd
#In order to show all columns available
pd.set_option('display.max_columns', 200)

#Sklearn imports
from sklearn.preprocessing import LabelEncoder, Imputer

#Graphing libs
import matplotlib.pyplot as plt
import seaborn as sns

apptrain = pd.read_csv('../Dataset/application_train.csv')
apptest = pd.read_csv('../Dataset/application_test.csv')

# Code that modifies dataframes
apptrain['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
apptest['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
apptrain['DAYS_BIRTH'] = abs(apptrain['DAYS_BIRTH'])
apptest['DAYS_BIRTH'] = abs(apptrain['DAYS_BIRTH'])

In [3]:
# One-hot encoding and dataframe alignment
le = LabelEncoder()
le_count = 0

# Iterate through columns
for col in apptrain:
    if apptrain[col].dtype == "object":
        if len(list(apptrain[col].unique())) <= 2:
            #train on the training data
            le.fit(apptrain[col])
            #transform both training and testing data
            apptrain[col] = le.transform(apptrain[col])
            apptest[col] = le.transform(apptest[col])
            
            le_count += 1
            

            
#One-Hot encoding
apptrain = pd.get_dummies(apptrain)
apptest = pd.get_dummies(apptest)



print('Training features shape: {}'.format(apptrain.shape))
print('Training features shape: {}'.format(apptest.shape))
print('{} columns were label encoded'.format(le_count))

Training features shape: (307511, 243)
Training features shape: (48744, 239)
3 columns were label encoded


In [4]:
# Take the labels out of the training dataset as an inner merge will erase them since the test dataset does not have the targets
train_labels = apptrain['TARGET']


#aligning the training and testing data, keep only columns present in both df's
apptrain, apptest = apptrain.align(apptest, join = 'inner', axis = 1)
apptrain['TARGET'] = train_labels

print('Training Features shape: ', apptrain.shape)
print('Testing features shape: ', apptest.shape)
print("We're back on track, remember the training dataset will have one column more since it DOES have the targets")

Training Features shape:  (307511, 240)
Testing features shape:  (48744, 239)
We're back on track, remember the training dataset will have one column more since it DOES have the targets


In [6]:
# Scaling not very much required for Random Forest models
# Creating base df's for machine learning model
training_data = apptrain.drop(columns = ['TARGET'])
testing_data = apptest.copy()

# In the dataframes we still have missing values, WE USE IMPUTATION HERE
imputer = Imputer(strategy = 'median')
imputer.fit(training_data)
imputer.fit(testing_data)
training_data = imputer.transform(training_data)
testing_data = imputer.transform(testing_data)

# We must scale the data as well.
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

training_data = scaler.fit_transform(training_data)
testing_data = scaler.transform(testing_data)


print('training data shape', training_data.shape)
print('testing data shape', testing_data.shape)

training data shape (307511, 239)
testing data shape (48744, 239)


# MLP Neural Network (BASE) model

In [7]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(activation='relu', learning_rate='adaptive', max_iter=800)
mlp.fit(training_data, train_labels)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=800, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [11]:
mlp_base_predictions = mlp.predict_proba(testing_data)[:, 1]

def format_and_submit(predictions, desired_file_name):
    submit = apptest[['SK_ID_CURR']]
    submit['TARGET'] = predictions
    submit.to_csv('../Model_Predictions/{}.csv'.format(desired_file_name), index = False)
    

format_and_submit(mlp_base_predictions, 'MLP_NN_Base_Predictions')

C:\Users\rGust\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
